In [ ]:

# Import necessary libraries
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score
import torch
import torchvision
from torchvision import transforms, models
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import pickle
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [ ]:

# Load CSV Files
train_df = pd.read_csv('cardatasettrain.csv')
test_df = pd.read_csv('cardatasettest.csv')

# Clean DataFrames
train_df_clean = train_df.drop(columns=['Unnamed: 0'])
test_df_clean = test_df.drop(columns=['Unnamed: 0'])


In [ ]:

# Define image loading function with consistent shape handling
def load_images(dataframe, folder_path, img_size=(64, 64)):
    images = []
    for img_name in dataframe['image']:
        img_path = os.path.join(folder_path, img_name)
        try:
            img = Image.open(img_path).resize(img_size).convert('RGB')  # Ensure RGB
            img_array = np.array(img).flatten()  # Flatten the image
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            continue
    return np.array(images, dtype=np.float32)

# Load training and testing images
train_images = load_images(train_df_clean, "cars_train/cars_train")
test_images = load_images(test_df_clean, "cars_test/cars_test")

# Verify loaded image shapes
print(f"Train Images Shape: {train_images.shape}")
print(f"Test Images Shape: {test_images.shape}")


In [ ]:

# Extract bounding box features
train_boxes = train_df_clean[['x1', 'y1', 'x2', 'y2']].values
test_boxes = test_df_clean[['x1', 'y1', 'x2', 'y2']].values

# Combine image features with bounding boxes
X_train_full = np.hstack((train_boxes, train_images))
X_test = np.hstack((test_boxes, test_images))

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_df_clean['Class'].values)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_encoded, test_size=0.2, random_state=42
)


In [ ]:

# Define ResNet-based model
class ResNetModel(nn.Module):
    def __init__(self, num_classes=196):
        super(ResNetModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)


In [ ]:

# Initialize model, criterion, optimizer, and data loaders
model = ResNetModel(num_classes=196).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Create TensorDatasets and DataLoaders
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Training Loop
num_epochs = 25
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects / len(train_loader.dataset)

    print(f'Epoch {epoch}/{num_epochs - 1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # Evaluate the model
    model.eval()
    val_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data).item()

    val_acc = val_corrects / len(val_loader.dataset)
    print(f'Validation Acc: {val_acc:.4f}')

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_resnet_model.pth')
print(f'Best Validation Accuracy: {best_acc:.4f}')


In [ ]:

# Evaluate the trained model
model.eval()
y_train_pred = []
y_val_pred = []

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.cuda()
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1).cpu().numpy()
        y_train_pred.extend(preds)

    for inputs, labels in val_loader:
        inputs = inputs.cuda()
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1).cpu().numpy()
        y_val_pred.extend(preds)

# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred, average='weighted')
val_precision = precision_score(y_val, y_val_pred, average='weighted')

# Print metrics
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
print(f'Validation F1 Score: {val_f1:.2f}')
print(f'Validation Precision: {val_precision:.2f}')

In [ ]:
# Save the trained model
model_filename = "fastai_resnet.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(model.state_dict(), file)

print(f"Model saved as {model_filename}")